In [ ]:
from refactor.dpo.model import DPOTrainModule

tac_gen_first = DPOTrainModule.load(
      'experiments/runs/lean_dojo_dpo/dpo_beamsearch_2023_12_05/10_09_00/checkpoints/epoch=0-step=10000-Pass@1_val=0.46.ckpt', device='cuda', freeze=True
)


In [ ]:
from models.iql_model import PerTokenIQL

In [ ]:
iql = PerTokenIQL(tac_gen_first.generator, tac_gen_first.tokenizer)

In [ ]:
# generate input to test

tokenizer = tac_gen_first.tokenizer


In [ ]:
state = 'E : Type u_3,\n' \
      '_inst_1 : normed_add_comm_group E,\n' \
      'f : ℝ → E,\n' \
      'a b c d : ℝ,\n' \
      'μ : measure ℝ,\n' \
      'h : [c, d] ⊆ [a, b],\n' \
      'hf : integrable_on f (Ι a b) μ,\n' \
      'hc : ¬c = d\n' \
      '⊢ integrable_on f (Ι c d) μ'

In [ ]:
tokenized_input = tokenizer(state, return_tensors="pt", max_length=2048, truncation=True)

In [ ]:
tokenized_input

In [ ]:
tokenized_input.input_ids.shape

In [ ]:
# tac_gen_first.generate(state, None, 1)

In [ ]:
tactic = 'have : <a>measure_theory.integrable_on</a> f (Ι c d) μ := hf.mono_set <a>set.Ioo_subset_uIoc_self</a>'

In [ ]:
tokenized_target=  tokenizer(tactic, return_tensors="pt", max_length=2048, truncation=True)

In [ ]:
output = tac_gen_first.generator(input_ids = tokenized_input.input_ids.cuda(), attention_mask = tokenized_input.attention_mask.cuda(), decoder_input_ids = tokenized_target.input_ids.cuda(), decoder_attention_mask = tokenized_target.attention_mask.cuda(), output_hidden_states=True)

In [ ]:
output.keys()

In [ ]:
output.decoder_hidden_states[-1].shape

In [ ]:
import torch
a = torch.randn(5,6,7)

In [ ]:

b = torch.randn(5,6,7)


In [ ]:
output.logits.shape

In [ ]:
len(output.encoder_hidden_states[0][0][0])

In [ ]:
# decoder gives hidden state tuple for each output token
len(output.decoder_hidden_states)

In [ ]:
tokenizer.decode().



In [ ]:
tactic = 'apply hf.mono_set'

In [ ]:
print (state)

In [ ]:
print (state + tactic)

In [ ]:
tokenized_input = tokenizer(state, return_tensors="pt", max_length=2048, truncation=True)

In [ ]:
tokenized_input